In [2]:
import pandas as pd
from contract_builder import get_data

flcci = pd.read_excel(R'C:\Users\PerdoCaiafa\Pontte-Servicer\IR\fluxocci.xlsx')
ids_flex = pd.read_excel(R'C:\Users\PerdoCaiafa\Pontte-Servicer\IR\ids_flex.xlsx')

normais = flcci.loc[flcci['Status do Contrato'] != 'ATIVO (QUITADO)']
normais = normais.loc[~normais['Identificador do Contrato'].isin(ids_flex['id'])]


normais = normais.rename(columns={
    'Operação': 'operacao',
    'Identificador do Contrato': 'id',
    'Mutuário': 'cliente',
    'Valor Recebido': 'val_recebido',
    'Valor Atual': 'val_atual',
    'VL Atual Calc Fat SCCI': 'val_atual_scci'

})

normais = normais[['id', 'operacao', 'cliente', 'val_recebido', 'val_atual', 'val_atual_scci']]


saldos = normais.groupby(by='id').agg({
    'operacao': 'first',
    'cliente': 'first',
    'val_recebido': 'sum',
    'val_atual': 'sum',
    'val_atual_scci': 'sum'
})

In [ ]:
pgtos = pd.DataFrame(
    columns= ['id', 'pago_total', 'pago2021']
    )


for id in saldos.index:

    try:

        print(id)
        new_df = pd.DataFrame(
        columns= ['id', 'pago_total', 'pago2021']
        )

        brute_data = get_data(id, "getRenegParcelas")
        parcelas =  pd.DataFrame(
        brute_data['data']['reneg']['parcelas']
        )


        allpaid = parcelas.loc[((parcelas['valor_pago'] != 'Paga') & (parcelas['id_renegociacao'] != ''))]
        totalpago = allpaid['valor_corrigido'].sum()

        allpaid['data_baixa'] = pd.to_datetime(allpaid['data_baixa'])

        paid2021 = allpaid.loc[allpaid['data_baixa'].dt.year == 2022].copy()
        totalpago2021 = paid2021['valor_corrigido'].sum()

        new_data = {
            'id': [id],
            'pago2021': [totalpago2021],
            'pago_total': [totalpago]
        }

        new_df = pd.DataFrame(data = new_data)
    
    except Exception:
        pass
    

    pgtos = pd.concat([pgtos, new_df]) 


In [10]:
df_test = pd.DataFrame(
    columns= ['id', 'pago_total', 'pago2021']
    )


raw_data = get_data(129282, "getRenegParcelas")

parcelas =  pd.DataFrame(
        raw_data['data']['reneg']['parcelas']
        )


allpaid = parcelas.loc[((parcelas['status_parcela'] == 'Paga') & (parcelas['id_renegociacao'] != ''))]
totalpago = allpaid['valor_corrigido'].sum()

allpaid.loc[:, 'data_baixa'] = pd.to_datetime(allpaid.loc[:, 'data_baixa'])


paid2021 = allpaid.loc[allpaid['data_baixa'].dt.year == 2022].copy()
totalpago2021 = paid2021['valor_corrigido'].sum()


new_data = {
            'id': 129282,
            'pago2021': totalpago2021,
            'pago_total': totalpago
        }





<ipython-input-10-da663bd33112>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  allpaid.loc[:, 'data_baixa'] = pd.to_datetime(allpaid.loc[:, 'data_baixa'])


In [3]:
pgtos

,id,pago_total,pago2021
0,129241,99180.87,27527.61
0,129243,53214.19,32925.84
0,129246,2186.11,0.0
0,129252,8158.82,0.0
0,129256,55092.65,37970.26
...,...,...,...
0,130198,18717.53,0.0
0,130206,1212.06,0.0
0,130209,10132.2,0.0
0,130211,0.0,0.0


In [3]:
raw_test = get_data(129246, 'getRenegParcelas')

In [4]:
testcelas =  pd.DataFrame(
        raw_test['data']['reneg']['parcelas']
        )

In [19]:
testcelas.to_excel('teste_ulele.xlsx')

In [20]:
testcelas.head(20)

,id_renegociacao,data_renegociacao,tipo_renegociacao,id_parcela,numero_parcela,data_vencimento,data_vencimento_boleto,data_baixa,data_credito,valor_principal,...,valor_cobrado,status_parcela,mensagem,indice,defasagem,data_base,grupo_boleto,periodicidade_correcao,id_renegocicao_base,id_renegocicao_resultado
0,320,2020-02-06T18:09:00.39,,176085,1/120 - Mensal,2020-02-05T00:00:00,0001-01-01T00:00:00,2020-02-06T00:00:00,2020-02-07T00:00:00,892.42,...,2145.53000,Paga,,IGP-M,2,2020-01-06T00:00:00,0,0,None,None
1,320,2020-02-06T18:09:01.52,,176091,001 - Seguro DFI,2020-02-05T00:00:00,0001-01-01T00:00:00,2020-02-06T00:00:00,2020-02-07T00:00:00,40.48,...,41.30000,Paga,,FIXO,2,2020-02-03T10:05:31.89,0,0,None,None
2,320,2020-02-06T18:09:01.417,,176089,001 - Seguro MIP,2020-02-05T00:00:00,0001-01-01T00:00:00,2020-02-06T00:00:00,2020-02-07T00:00:00,17.85,...,18.21000,Paga,,FIXO,2,2020-02-03T10:05:31.16,0,0,None,None
3,320,2020-02-06T18:09:01.287,,176087,1/120 - Taxa de Adm.,2020-02-05T00:00:00,0001-01-01T00:00:00,2020-02-06T00:00:00,2020-02-07T00:00:00,25.00,...,25.50000,Paga,,FIXO,2,2020-01-06T00:00:00,0,0,None,None
4,,0001-01-01T00:00:00,,173770,2/120 - Mensal,2020-03-05T00:00:00,2020-03-05T00:00:00,2020-03-06T07:00:43.303,2020-03-06T00:00:00,908.65,...,2138.79000,Paga,,IGP-M,2,2020-01-06T00:00:00,0,0,None,None
5,,0001-01-01T00:00:00,,177552,0 - Seguro DFI,2020-03-05T00:00:00,2020-03-05T00:00:00,2020-03-06T07:00:43.303,2020-03-06T00:00:00,41.30,...,41.30000,Paga,,FIXO,2,2020-03-01T00:00:00,0,0,None,None
6,,0001-01-01T00:00:00,,177550,0 - Seguro MIP,2020-03-05T00:00:00,2020-03-05T00:00:00,2020-03-06T07:00:43.303,2020-03-06T00:00:00,23.54,...,23.54000,Paga,,FIXO,2,2020-03-01T00:00:00,0,0,None,None
7,,0001-01-01T00:00:00,,177551,0 - Taxa de Adm.,2020-03-05T00:00:00,2020-03-05T00:00:00,2020-03-06T07:00:43.303,2020-03-06T00:00:00,25.00,...,25.00000,Paga,,FIXO,2,2020-03-01T00:00:00,0,0,None,None
8,4448,2021-10-26T13:32:41.75,12,723835,1 - Mensal,2020-05-05T00:00:00,0001-01-01T00:00:00,2021-10-26T16:51:17.137,2020-05-06T00:00:00,1146.44,...,1146.44000,Quebra Promessa,,IGP-M,2,2020-05-05T00:00:00,0,0,None,None
9,4448,2021-10-26T13:32:41.75,12,723836,1 - Mensal,2021-01-05T00:00:00,0001-01-01T00:00:00,2021-10-26T16:58:44.793,2021-01-19T00:00:00,1177.20,...,3514.54000,Quebra Promessa,,IGP-M,2,2020-06-05T00:00:00,0,0,None,None


In [11]:
raw_test2 = get_data(129246, 'getFinances')

testcelas2 =  pd.DataFrame(
        raw_test2['data']['posicaofinanceira']['parcelas']
        )

In [12]:
testcelas2

,data_vencimento,valor_principal,valor_juros,valor_original,valor_correcao,valor_corrigido,valor_multa,valor_mora,valor_desconto,valor_mip,valor_dfi,valor_taxa_adm,valor_cobrado,data_pagamento,valor_pago,valor_saldo,juros_incorrido,correcao_proporcional
0,2020-02-05T00:00:00,892.42,1167.27,2059.69,43.09,2102.78,43.70,0.73,0.00,17.85,40.48,25,2230.54,2020-02-05T00:00:00,2230.04,0.00,0.0000,-892.4200
1,2020-03-05T00:00:00,908.65,1178.61,2087.26,51.53,2138.79,0.00,0.00,0.00,23.54,41.30,25,2228.63,2020-03-04T00:00:00,2228.63,0.00,0.0000,-908.6500
2,2020-05-05T00:00:00,1146.44,0.00,1146.44,474.35,1620.79,0.00,0.00,474.35,0.00,0.00,0,1146.44,2020-05-05T00:00:00,1146.44,0.00,0.0000,-1146.4400
3,2021-01-05T00:00:00,1177.20,1328.70,2505.90,1008.64,3514.54,61.59,13.35,74.93,31.17,41.30,25,3612.02,2021-01-18T00:00:00,3612.01,0.00,0.0000,-1177.2000
4,2021-02-05T00:00:00,1177.20,1316.05,2493.25,1034.62,3527.87,72.51,460.42,0.00,31.19,41.30,25,4158.29,0001-01-01T00:00:00,0.00,175649.93,169.4231,558.8055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,2029-05-17T00:00:00,1177.20,63.27,1240.47,514.76,1755.23,0.00,0.00,0.00,1.77,41.30,25,1823.30,0001-01-01T00:00:00,0.00,8418.04,169.4231,558.8055
104,2029-06-17T00:00:00,1177.20,50.62,1227.82,509.50,1737.32,0.00,0.00,0.00,1.41,41.30,25,1805.03,0001-01-01T00:00:00,0.00,6734.43,169.4231,558.8055
105,2029-07-17T00:00:00,1177.20,37.96,1215.16,504.26,1719.42,0.00,0.00,0.00,1.06,41.30,25,1786.78,0001-01-01T00:00:00,0.00,5050.82,169.4231,558.8055
106,2029-08-17T00:00:00,1177.20,25.31,1202.51,499.00,1701.51,0.00,0.00,0.00,0.71,41.30,25,1768.52,0001-01-01T00:00:00,0.00,3367.21,169.4231,558.8055
